## Setup

In [1]:
import os
import itertools

import pandas as pd
import numpy as np

## Decision Trees

We will use obesity data from kaggle; you may access it by clicking on this download [link](https://www.kaggle.com/datasets/yersever/500-person-gender-height-weight-bodymassindex?resource=download) if you would wish to see context (make sure you have an account with kaggle). If you don't have a kaggle account, I have uploaded the dataset in the root directory lecture for your convenience.


In our particular example, we will simplify our task to a binary classification.

In [2]:
# dir path
path_to_dir = "../sample"
filename = "500_Person_Gender_Height_Weight_Index.csv"

filepath = os.path.join(path_to_dir, filename)
filepath

'../sample/500_Person_Gender_Height_Weight_Index.csv'

In [3]:
df = pd.read_csv(filepath)

In [4]:
df.head()

,Gender,Height,Weight,Index
0,Male,174,96,4
1,Male,189,87,2
2,Female,185,110,4
3,Female,195,104,3
4,Male,149,61,3


In [5]:
# some descriptive stats for each field
df.describe()

,Height,Weight,Index
count,500.000000,500.000000,500.000000
mean,169.944000,106.000000,3.748000
std,16.375261,32.382607,1.355053
min,140.000000,50.000000,0.000000
25%,156.000000,80.000000,3.000000
50%,170.500000,106.000000,4.000000
75%,184.000000,136.000000,5.000000
max,199.000000,160.000000,5.000000


### Target variable

In [6]:
# obese

df.loc[:, "is_obese"] = (df["Index"] >= 4).astype(int)
df.sample(10)

,Gender,Height,Weight,Index,is_obese
322,Male,153,104,5,1
201,Male,179,93,3,0
356,Female,189,124,4,1
329,Female,185,102,3,0
182,Male,162,97,4,1
213,Female,196,159,5,1
465,Male,158,127,5,1
464,Female,181,80,2,0
75,Female,197,154,4,1
221,Male,183,138,5,1


In [7]:
# we drop the Index column

df.drop("Index", axis=1, inplace=True)

### Gini impurity and entropy

Impurity measures the degree to which a partition or cut (splitting or branching) will classify or predict the target variable **incorrectly**. In the case of binary classification, which is our running example, that would measure whether the target values 0 and 1 are classified correctly.

In this sense, a zero impure split or 100 percent split will get all our predictions correct. The smaller the impurity measure, the more important the variable (where the split happens) is.


In [8]:
# impurity

# in this example we look at the following partitions
# partition 1: Weight at 100 kg (anything >= 100 should be marked as obese)

partition1 = df["Weight"] >= 100  # this is a mask
incorrect_label = df["is_obese"] == 0  # also a mask

partition1_obese_count = df[partition1 & incorrect_label].shape[0]


# partition 2: Weight at 70 kg (similar to above)
partition2 = df["Weight"] >= 70

partition2_obese_count = df[partition2 & incorrect_label].shape[0]

print(f"Partition 1 incorrect predictions: {partition1_obese_count}")
print(f"Partition 2 incorrect predictions: {partition2_obese_count}")

print("\nThe second parition is more impure!")

Partition 1 incorrect predictions: 18
Partition 2 incorrect predictions: 90

The second parition is more impure!


If we wish to compute the overall impurity for all cuts, we consider the **Gini** index. This is our cost function.

$$\text{Gini} = 1 - \sum_{i=1}^n p_i^2$$

Here $p_i$ is the probability of a class taking on a specific value.

It is the probability of misclassifying a certain feature.

One way to think about of the cost function above is to consider a fair and unfair coin random variable (a binary class label). If we have a fair coin with probability 0.5 (landing heads) then the index will be 0.5; it's as impure as we can get. But if we have a one-sided coin, then the index is 0. So there is no uncertainty or entropy. To describe uncertainty, we often say it's a flip of a coin! Yes indeed 0.5 is a flip of a fair coin.

In [9]:
# we implement the gini index as follows

def get_gini_impurity(y):
    """Compoute index of classification

        Args:
            y (pd.Series): classifcation values
    """

    # first check we have a series
    # if not throw an exception
    if not isinstance(y, pd.Series):
        raise Exception("Input must be a pd.Series object")

    # calculate individual probabilities (i.e. P_i)
    p = y.value_counts() / y.shape[0]

    # gini impurity
    gini = 1 - np.sum(p**2)

    return gini



# example: Gender
get_gini_impurity(df["Gender"])

# NOTE: if our feature has more than one unique value
#       impurity will range between 0 and 1

0.4998

### Entropy

Similarly, entropy measures impurity by considering the uncertainty of a certain outcome. Entropy, regardless of the possible number of unique values a class variable can take on, ranges between 0 and 1. The greater the entropy, the more uncertain the variable outcome is.

$$\text{Entropy} = - \sum_{i=1}^n p_i \log_2 p_i $$



In [10]:
# entropy implementation

def get_entropy(y):
    """Compute entropy given series y
    """

    # first check we have a series
    # if not throw an exception
    if not isinstance(y, pd.Series):
        raise Exception("Input must be a pd.Series object")

    # calculate individual probabilities (i.e. P_i)
    p = y.value_counts() / y.shape[0]

    # entropy
    epsilon = 1e-9  # small number
                    # to avoid taking log of zero
    entropy = np.sum(
        -p * np.log2(p + epsilon)  # broadcast
    )

    return entropy

# example
get_entropy(df["Gender"])

0.9997114388674198

### Which cuts?

So how do we find the optimal cuts that give us the lowest entropy or gini impurity. We rephrase this question in terms of impurity loss. If we experience a decrease in uncertainty, then the cut has improved.

$$IG = \text{Impurity} (\text{parent})  - \sum_{\text{child}} \text{prop}_{\text{child}} \times \text{Impurity}(\text{child})$$ 

A few comments. For classification tasks, impurity function chosen is entropy. While, for regression tasks, the function is variance.

##### Information Gain

In [11]:
# implementation

# helper function

# variance
def get_var(y):
    """Compute variance of random variable

        Args:
            y (pd.Series): random variable values
    """

    if len(y) == 1: return 0

    return y.var()


# information gain
def get_ig(y, mask, impurity=get_entropy):
    """Compute IG for a single child split
    """

    prop = sum(mask) / len(mask)

    parent_impurity = get_entropy(y)
    

    # child1_impurity = prop * get_entropy(df["is_obese"][mask])
    # child2_impurity = (1 - prop) * get_entropy(df["is_obese"][~mask])


    child1_impurity = prop * get_entropy(y[mask])
    child2_impurity = (1 - prop) * get_entropy(y[~mask])

    ig = parent_impurity - (child1_impurity + child2_impurity)


    return ig

# example
get_ig(df["is_obese"], df["Gender"]=="Male")


0.0005506911187600494

##### The best spilt for a variable

In [12]:
# numerical values are totally ordered

# but for categorical we create masks by inclusion
# this means creating all possible combination of subsets


def generate_all_subsets(y):
    """Generate all possible subsets rand var values

    Args:
        y (pd.Series): random variable
    """

    y_unique = y.unique()

    subsets = []

    for subset_len in range(0, len(y_unique)+1):
        for subset in itertools.combinations(y_unique, subset_len):
            subset_list = list(subset)

            subsets.append(subset_list)

    return subsets


# example
generate_all_subsets(df["Gender"])   

[[], ['Male'], ['Female'], ['Male', 'Female']]

In [13]:
# max ig split

def get_max_ig_split(x, y, impurity=get_entropy):
    """Compute the mask boundary with greatest information gain
    Args:
        x (pd.Series): predictor variable
        y (pd.Series): target variable
        impurity (func): entropy or variance

    """

    is_numeric = True if x.dtypes != "O" else False
    
    ig_values = []
    split_values = []


    if is_numeric:
        options = x.sort_values().unique()[1: ] # we skip the first
    # categorical
    else:
        options = generate_all_subsets(x)

    for option in options:
        mask = x < option if is_numeric else x.isin(option)

        # compute ig
        option_ig = get_ig(y, mask, impurity)

        ig_values.append(option_ig)
        split_values.append(option)
    # if there are no results

    if not ig_values:
        return None, None, None, False

    max_ig = max(ig_values)
    max_ig_index = ig_values.index(max_ig)
    best_split = split_values[max_ig_index]


    return max_ig, best_split, is_numeric, True
    

# example
get_max_ig_split(df["Gender"], df["is_obese"])


(0.0005506911187600494, ['Male'], False, True)

In [14]:
ig_df = df.drop("is_obese", axis=1).apply(get_max_ig_split, y=df["is_obese"])

ig_df.rename(
    index=dict(
        zip(
            list(range(0,4)), ["max_ig", "split_val", "is_numeric", "has_ig"]
        )
    ), inplace=True
)

ig_df

,Gender,Height,Weight
max_ig,0.000551,0.064748,0.382454
split_val,[Male],174,103
is_numeric,False,True,True
has_ig,True,True,True


In [15]:
# write helper function to get best split
# ie predictor with highes information gain

def get_best_split(target_label, data):

    ig_df = data.drop(
        "is_obese", axis=1
    ).apply(
        get_max_ig_split, y=data[target_label]
    )

    ig_df.rename(
        index=dict(
            zip(
                list(range(0,4)), ["max_ig", "max_ig_index", "best_split", "has_ig"]
            )
        ), inplace=True
    )

    # first check whether an ig has computed

    # take the last row and compute the sum

    if sum(ig_df.iloc[-1, :]) == 0: return None, None, None, None

    best_feature = max(ig_df)

    ig, split_index, split_value = ig_df[best_feature].values[:3]

    return best_feature, split_index, ig, split_value


def make_split(variable, split_value, data, is_numeric):
    """Split data given boundary or subset
    """

    if is_numeric:
        mask = data[variable] < split_value
    else:
        mask = data[variable].isin(split_value)

    return data[mask], data[~mask]


def make_prediction(ser, is_numeric):

    if is_numeric: return ser.mean()

    return ser.value_counts().idxmax()

# # test
# get_best_split("is_obese", df)
# make_split("Weight", 100, df, True)[1].shape
# make_prediction(df["Gender"], False)
# df[df.Gender=="Female"].shape[0] > df[df.Gender=="Male"].shape[0]

#### Build tree

We will not consider the full gamut of hyperparemeters we find in sklearn when training decision trees but will include the following: `min_samples_split`, `max_depth`, and `min_information_gain`.

In [16]:
# Finally, we are ready to build our decision tree
# the output will be a nested dictionary

def build_tree(
    data,
    target_var,
    is_target_categorical,
    max_depth=None,
    min_samples_split=None,
    min_information_gain=1e-10,
    counter=0,  # once a split happens, depth increases by 1 and max cat checked
    max_categories=20
):

    # check max categories
    if counter==0:
        types = data.dtypes
        check_columns = types[types=="object"].index

        cat_df = data[check_columns]
        unique_val_ser = cat_df.apply(lambda x: len(x.unique()))

        if sum(unique_val_ser > max_categories) > 0:
            raise Exception("Number of categories is too many!")

    # check max depth
    if max_depth == None:
        depth_cond = True
    else: # check counter
        depth_cond = True if counter < max_depth else False
    
    # check min samples
    if min_samples_split == None:
        sample_cond = True
    else:
        sample_cond = True if data.shape[0] > min_samples_split else False

    
    # if depth and min_samples are a check
    # check ig condition is met

    if depth_cond and sample_cond:
        var, split_val, ig, is_numeric = get_best_split(target_var, data)

        # check ig condition
        if ig is not None and ig >= min_information_gain:
            counter += 1 # depth increase
            # split data
            left, right = make_split(var, split_val, data, is_numeric)

            # create a subtree
            split_type = "<=" if is_numeric else "in"
            condition = f"{var} {split_type}  {split_val}"
            subtree = {condition: []}

            # find splits/leaves (via recursion)
            obese_pos = build_tree(
                left,
                target_var,
                is_target_categorical,
                max_depth,
                min_samples_split,
                min_information_gain,
                counter
            )

            obese_neg = build_tree(
                right,
                target_var,
                is_target_categorical,
                max_depth,
                min_samples_split,
                min_information_gain,
                counter
            )

            # the only time obese_pos equals obese_neg
            # is when you have already reached a leaf
            # and there is a prediction
            if obese_pos == obese_neg: 
                subtree = obese_pos
            # otherwise, we have subtrees
            else:
                subtree[condition].append(obese_pos)
                subtree[condition].append(obese_neg)
        # found a leaf
        else:
            return make_prediction(
                data[target_var],
                not is_target_categorical
            )
    # depth or min_samples condition is not met
    else:
        return make_prediction(
                data[target_var],
                not is_target_categorical
            )

    return subtree


max_depth = 5
min_samples_split = 20
min_information_gain  = 1e-5

dt = build_tree(
    df,
    "is_obese",
    True,
    max_depth,
    min_samples_split,
    min_information_gain
)


dt

{'Weight <=  103': [{'Weight <=  66': [0,
    {'Weight <=  84': [{'Weight <=  74': [0, {'Weight <=  75': [1, 0]}]},
      {'Weight <=  98': [1, 0]}]}]},
  1]}

#### Evaluate

In [17]:
# helper function to predict single observation

def pred_obs(observation, tree):
    """Output tree prediction on observation
    """

    # start at the root
    cond = list(tree.keys())[0]

    feature, split_type, split_val = cond.split()

    if split_type == "<=":
       # next, we check whether to traverse the further
       # check whether observations meets condition
       # splitting into subtrees
       answer = tree[cond][0] if observation[feature] <= float(split_val) else tree[cond][1]

    # if leaf, return value
    if not isinstance(answer, dict):
        return answer

    # traverse recursively
    return pred_obs(observation, answer)


In [18]:
# train accuracy

num_obs = df.shape[0]
preds = []
for i in range(num_obs):
    obs_pred = pred_obs(df.iloc[i,:], dt)
    preds.append(obs_pred)

acc = (np.array(preds) == df["is_obese"]).mean()

print(f"train accuracy: {acc}")

train accuracy: 0.848
